In [ ]:
!pip install bokeh
import bokeh
bokeh.sampledata.download()

In [3]:
import panel as pn
import pandas as pd
import altair as alt

from bokeh.sampledata import stocks

pn.extension('vega', template='fast-list')

This example is meant to make it easy to compare and contrast the different APIs Panel provides to declare apps and dashboards. Specifically, it compares four different implementations of the same app using 1) the quick and easy ``interact`` function, 2) more flexible reactive functions, 3) declarative Param-based code, and 4) explicit callbacks.

Before comparing the different approaches, we will first declare some components of the app that will be shared, including the title of the app, a set of stock tickers, a function to return a dataframe given the stock ``ticker`` and the rolling mean ``window_size``, and another function to return a plot given those same inputs:

In [4]:
test_df = pd.DataFrame(getattr(stocks,'AAPL'))
getattr(test_df, 'close')

0       130.31
1       122.00
2       128.00
3       125.69
4       122.87
         ...  
3265    442.80
3266    448.97
3267    444.57
3268    441.40
3269    430.47
Name: close, Length: 3270, dtype: float64

In [5]:
brush = alt.selection_interval(name='brush')

@pn.depends(vega_pane.selection.param.brush)
def filtered_table(selection):
    if not selection:
        return '## No selection'

    query = ' & '.join(
        f'{values[0]} <= `{col}` <= {values[1]}'
        if pd.api.types.is_numeric_dtype(df[col])
        else f'`{col}` in {values}' 
        for col, values in selection.items()
    )
    return pn.Column(
        f'Query: {query}', selection.items(),
        pn.pane.DataFrame(df.query(query), width=600, height=300)
    )

NameError: name 'vega_pane' is not defined

In [6]:
title = '## Stock Explorer Altair'

tickers = ['AAPL', 'FB', 'GOOG', 'IBM', 'MSFT']


def get_df(ticker, window_size):
    df = pd.DataFrame(getattr(stocks, ticker))
    df['date'] = pd.to_datetime(df.date)
    return df.set_index('date').rolling(window=window_size).mean().reset_index()

def get_plot(ticker, window_size):
    df = get_df(ticker, window_size)
    return alt.Chart(df).mark_line().encode(x='date', y='close').properties(width="container", height="container").add_selection(brush)


### Interact

In the ``interact`` model the widgets are automatically generated from the arguments to the function or by providing additional hints to the ``interact`` call. This is a very convenient way to generate a simple app, particularly when first exploring some data.  However, because widgets are created implicitly based on introspecting the code, it is difficult to see how to modify the behavior.  Also, to compose the different components in a custom way it is necessary to unpack the layout returned by the ``interact`` call, as we do here:

In [7]:
interact = pn.interact(get_plot, ticker=tickers, window_size=(1, 21, 5))
interact
# pn.Row(
#     pn.Column(title, interact[0]),
#     interact[1]
# )

Column
    [0] Column
        [0] Select(name='ticker', options=['AAPL', 'FB', ...], value='AAPL')
        [1] IntSlider(end=21, name='window_size', start=1, step=5, value=11)
    [1] Row
        [0] Vega(Chart, name='interactive00101', selection=Selection)

### Reactive

The reactive programming model is similar to the ``interact`` function but relies on the user (a) explicitly instantiating widgets, (b) declaring how those widgets relate to the function arguments (using the ``bind`` function), and (c) laying out the widgets and other components explicitly. In principle we could reuse the ``get_plot`` function from above here but for clarity we will repeat it:

In [8]:
brush = alt.selection_interval(name='brush')

ticker = pn.widgets.Select(name='Ticker', options=tickers)
window = pn.widgets.IntSlider(name='Window Size', value=6, start=1, end=21)

def get_df(ticker, window_size):
    df = pd.DataFrame(getattr(stocks, ticker))
    df['date'] = pd.to_datetime(df.date)
    return df.set_index('date').rolling(window=window_size).mean().reset_index()

def get_plot(ticker, window_size):
    df = get_df(ticker, window_size)
    return pn.pane.Vega(alt.Chart(df).mark_line().encode(x='date', y='close').add_selection(brush).properties(
        width="container", height="container" ), debounce=5)




def filtered_table(selection):
    if not selection:
        return '## No selection'
    query = ' & '.join(
        f'{values[0]} <= `{col}` <= {values[1]}'
        if pd.api.types.is_numeric_dtype(df[col])
        else f'`{col}` in {values}' 
        for col, values in selection.items()
    )
    return pn.Column(
        f'Query: {query}', selection.items(),
        pn.pane.DataFrame(df.query(query), width=600, height=300)
    )

pn.Row(
    pn.Column(title, ticker, window),
    pn.bind(get_plot, ticker, window)
    ,
    # pn.bind(filtered_table, [1,2])
   
)



Row
    [0] Column
        [0] Markdown(str)
        [1] Select(name='Ticker', options=['AAPL', 'FB', ...], value='AAPL')
        [2] IntSlider(end=21, name='Window Size', start=1, value=6)
    [1] ParamFunction(function, _pane=Vega)

In [115]:
ticker.value

'GOOG'

### Parameterized class

Another approach expresses the app entirely as a single ``Parameterized`` class with parameters to declare the inputs, rather than explicit widgets. The parameters are independent of any GUI code, which can be important for maintaining large codebases, with parameters and functionality defined separately from any GUI or panel code. Once again the ``depends`` decorator is used to express the dependencies, but in this case the dependencies are expressed as strings referencing class parameters, not parameters of widgets. The parameters and the ``plot`` method can then be laid out independently, with Panel used only for this very last step.

In [251]:
import param

# brush = alt.selection_interval(name='brush')
def get_df(ticker, window_size):
    df = pd.DataFrame(getattr(stocks, ticker))
    df['date'] = pd.to_datetime(df.date)
    return df.set_index('date').rolling(window=window_size).mean().reset_index()

def get_plot(ticker, window_size):
    df = get_df(ticker, window_size)
    return alt.Chart(df).mark_line().encode(x='date', y='close').add_selection(brush).properties(width="container", height="container")


class StockExplorer(param.Parameterized):
    
    ticker = param.Selector(default='AAPL', objects=tickers)
    window_size = param.Integer(default=6, bounds=(1, 21))
    
    @param.depends('ticker', 'window_size')
    def plot(self):
        
        return get_plot(self.ticker, self.window_size).properties(width=600, height=300)
     

explorer = StockExplorer()

pn.Row(
    pn.Column(explorer.param),
    explorer.plot,
    
)

Row
    [0] Column
        [0] Column(margin=5, name='StockExplorer', width=300)
            [0] StaticText(value='<b>StockExplorer</b>')
            [1] Select(name='Ticker', options=OrderedDict([('AAPL', ...]), value='AAPL')
            [2] IntSlider(end=21, name='Window size', start=1, value=6)
    [1] ParamMethod(method, _pane=Vega)

In [245]:
vega_pane.selection.param.brush

### Callbacks

The above approaches are all reactive in some way, triggering actions whenever manipulating a widget causes a parameter to change, without users writing code to trigger callbacks explicitly.  Explicit callbacks allow complete low-level control of precisely how the different components of the app are updated, but they can quickly become unmaintainable because the complexity increases dramatically as more callbacks are added. The approach works by defining callbacks using the ``.param.watch`` API that either update or replace the already rendered components when a watched parameter changes:

In [72]:
ticker = pn.widgets.Select(name='Ticker', options=['AAPL', 'FB', 'GOOG', 'IBM', 'MSFT'])
window = pn.widgets.IntSlider(name='Window', value=6, start=1, end=21)

row = pn.Row(
    pn.Column(title, ticker, window, sizing_mode="fixed", width=300),
    get_plot(ticker.options[0], window.value)
)

def update(event):
    row[1].object = get_plot(ticker.value, window.value)

ticker.param.watch(update, 'value')
window.param.watch(update, 'value')

row

Row
    [0] Column(sizing_mode='fixed', width=300)
        [0] Markdown(str)
        [1] Select(name='Ticker', options=['AAPL', 'FB', ...], value='AAPL')
        [2] IntSlider(end=21, name='Window', start=1, value=6)
    [1] Vega(Chart, selection=Selection)

In practice, different projects will be suited to one or the other of these APIs, and most of Panel's functionality should be available from any API.

## App

This notebook may also be served as a standalone application by running it with `panel serve stocks_altair.ipynb`. Above we enabled a custom `template`, in this section we will add components to the template with the `.servable` method:

In [13]:
ticker.servable(area='sidebar')
window.servable(area='sidebar')

pn.panel("""This example compares **four different implementations of the same app** using 

- the quick and easy ``interact`` function, 
- more flexible *reactive* functions,
- declarative *Param-based* code, and 
- explicit *callbacks*.""").servable()

pn.panel(pn.bind(get_plot, ticker, window)).servable(title='Altair Stock Explorer');

In [9]:
import panel as pn
import pandas as pd
import altair as alt
pn.extension('vega', template='fast-list')

penguins_url = "https://raw.githubusercontent.com/vega/vega/master/docs/data/penguins.json"
df = pd.read_json(penguins_url)

brush = alt.selection_interval(name='brush')  # selection of type "interval"
island = pn.widgets.Select(name='Island', options=df.Island.unique().tolist())

chart = alt.Chart(df.query(f'Island == "{island.value}"')).mark_point().encode(
    x=alt.X('Beak Length (mm):Q', scale=alt.Scale(zero=False)),
    y=alt.Y('Beak Depth (mm):Q', scale=alt.Scale(zero=False)),  
    color=alt.condition(brush, 'Species:N', alt.value('lightgray'))
).properties(
    width=700,
    height=200
).add_selection(brush)

vega_pane = pn.pane.Vega(chart, debounce=5)

def filtered_table(selection, island):    
    if not selection:
        return '## No selection'
    query = ' & '.join(
        f'{crange[0]:.3f} <= `{col}` <= {crange[1]:.3f} & Island == "{island}"'
        for col, crange in selection.items()
    )
    return pn.Column(
        f'Query: {query}',
        pn.pane.DataFrame(df.query(query).query(f'Island == "{island}"'), width=600, height=300)
    )

pn.Column(
    pn.Row(island, vega_pane),
    pn.bind(filtered_table, selection = vega_pane.selection.param.brush, island=island))

Column
    [0] Row
        [0] Select(name='Island', options=['Torgersen', ...], value='Torgersen')
        [1] Vega(Chart, debounce=5, selection=Selection)
    [1] ParamFunction(function, _pane=Markdown)

In [ ]:
# island.value
df.query(f'Island == "{island.value}"')
# df.query(f'Island == "Torgersen"')
"33.579 <= Beak Length (mm) <= 41.755 & 18.699 <= Beak Depth (mm) <= 21.543" + f' & Island == "{island}"'
chart
island_selection.options[0]
(island_selection.controls(jslink=True), island_selection)

In [508]:
import pandas as pd
import param
penguins_url = "https://raw.githubusercontent.com/vega/vega/master/docs/data/penguins.json"
df = pd.read_json(penguins_url)

brush = alt.selection_interval(name='brush')  # selection of type "interval"
island_selection = pn.widgets.Select(name='Island', options=df.Island.unique().tolist(), value='Biscoe')

@pn.depends(island_selection.param.value)
def chart(island):
    print("1", island_selection.value)
    return alt.Chart(df.query(f'Island == "{island_selection.value}"')).mark_point().encode(
    x=alt.X('Beak Length (mm):Q', scale=alt.Scale(zero=False)),
    y=alt.Y('Beak Depth (mm):Q', scale=alt.Scale(zero=False)),
    color=alt.condition(brush, 'Species:N', alt.value('lightgray'))
    ).properties(width=700, height=200
    ).add_selection(brush
    )

vega_pane = pn.pane.Vega(chart(island_selection.value), debounce=5)

@pn.depends(vega_pane.selection.param.brush, island_selection.param.value)
def filtered_table(selection, island):
    if not selection:
        return "## No selection"
    query = " & ".join(
        f"{crange[0]:.3f} <= `{col}` <= {crange[1]:.3f} & Island == '{island_selection.value}'"
        for col, crange in selection.items()
    )
    return pn.Column(
        f'Query: {query}',
        pn.pane.DataFrame(df.query(query).query(f"Island == '{island_selection.value}'"), width=600, height=300)
    ) 


row = pn.Column(
    pn.Row(island_selection),
    pn.Row(vega_pane),
    pn.Row(filtered_table))
def update2(event=None):
    row[1].object = vega_pane = pn.pane.Vega(chart(island_selection.value), debounce=5)

island_selection.param.watch(update2, "value")
row

    # pn.bind(filtered_table, selection = vega_pane.selection.param.brush, island=island_selection) )
    
# ticker = pn.widgets.Select(name='Ticker', options=['AAPL', 'FB', 'GOOG', 'IBM', 'MSFT'])
# window = pn.widgets.IntSlider(name='Window', value=6, start=1, end=21)

# row = pn.Row(
#     pn.Column(title, ticker, window, sizing_mode="fixed", width=300),
#     get_plot(ticker.options[0], window.value)
# )

# def update(event):
#     row[1].object = get_plot(ticker.value, window.value)

# ticker.param.watch(update, 'value')
# window.param.watch(update, 'value')

# row

1 Biscoe


1 Dream 
1 Torgersen

Column
    [0] Row
        [0] Select(name='Island', options=['Torgersen', ...], value='Biscoe')
    [1] Row
        [0] Vega(Chart, debounce=5, selection=Selection)
    [2] Row
        [0] ParamFunction(function, _pane=Markdown)

In [504]:
df.query("Island == ‘Torgersen’")

SyntaxError: invalid character '‘' (U+2018) (<unknown>, line 1)

In [12]:
# standardizing state and county column names

apple_counties = pd.read_csv('./data/apple_clean_counties.csv', parse_dates = ['date'])
google_counties = pd.read_csv('./data/google_clean_counties.csv', parse_dates = ['date'])

apple_counties['state'] = apple_counties['sub_region']
apple_counties['county'] = apple_counties['region']

google_counties['state'] = google_counties['sub_region_1']
google_counties['county'] = google_counties['sub_region_2']

state_county_combinations = []
for i in list(zip(apple_counties.state, apple_counties.county)):
    state_county_combinations.append(', '.join(i))

state_county_combinations = set(state_county_combinations)   

case_data = pd.read_csv('data/jhu-case-data.csv', parse_dates= ['date'])

,date,type,volume
0,2020-02-15,retail_recreation,4.0
1,2020-02-16,retail_recreation,7.0
2,2020-02-17,retail_recreation,11.0
3,2020-02-18,retail_recreation,4.0
4,2020-02-19,retail_recreation,3.0


In [14]:

state_input = pn.widgets.Select(name = 'Select a state, county',
                                        options = list(state_county_combinations),
            
                                           placeholder = 'ex: Maryland, Calvert County',
                                          value = 'Maryland, Calvert County')

@pn.depends(state_input.param.value)
def state_county_plot(state_input):
    state, county = state_input.split(', ')

    apple_to_plot_sc = apple_counties[(apple_counties.state == state) & (apple_counties.county == county)]
    google_to_plot_sc = google_counties[(google_counties.state == state) & (google_counties.county == county)]

    apple_to_plot_sc = apple_to_plot_sc.melt(
        id_vars='date',
        value_vars = ['driving', 'transit', 'walking'],
        var_name = 'type',
        value_name = 'volume'
    )

    google_cols_to_melt_sc = google_to_plot_sc.columns[4:]
    google_to_plot_sc = google_to_plot_sc.melt(
        id_vars = 'date',
        value_vars = google_cols_to_melt_sc,
        var_name = 'type',
        value_name = 'volume'
    )


    brush = alt.selection_interval(encodings=['x'])

    color = alt.condition(brush,
                          alt.Color('type:Q', legend=None),
                          alt.value('lightgray'))


    apple_sc = apple = alt.Chart(apple_to_plot_sc).mark_line().encode(
        x = 'date:T',
        y = 'volume:Q',
        color = 'type:N'
    ).add_selection(brush).properties(title = 'Apple mobility data')

    google_sc = google = alt.Chart(google_to_plot_sc).mark_line().encode(
        x = alt.X('date:T', scale = alt.Scale(domain = brush)),
        y = 'volume:Q',
        color = 'type:N'
    ).properties(title = 'Google mobility data')
    
    subtitle = f"### Mobility and case data for {state}, {county}"
    
    
    county_to_plot = pd.DataFrame(
        case_data[(case_data.Province_State == state) & (case_data.Admin2 == county.split()[0])]
    )
    county_to_plot['new_cases'] = county_to_plot['cases'].rolling(window=2).apply(lambda x: x[1] - x[0], raw = True)
    
    
    county_cum_cases = alt.Chart(county_to_plot).mark_line().encode(
        x = alt.X('date:T', scale = alt.Scale(domain = brush), title = 'Cumulative Cases'),
        y = 'cases:Q'
    ).properties(
        title = {'text': 'Daily New Cases', 
                    'subtitle': 'Source: JHU'}
    )

    county_new_cases = alt.Chart(county_to_plot).mark_line().encode(
        x = alt.X('date:T', scale = alt.Scale(domain = brush), title = 'Daily New Cases'),
        y = 'new_cases:Q'
    ).properties(
        title = {'text': 'Daily New Cases', 
                    'subtitle': 'Source: JHU'}
    )


    county_plots_set = alt.vconcat(apple_sc | google_sc, county_cum_cases | county_new_cases)
    
    return pn.Column(subtitle, county_plots_set)



state_county_dash = pn.Row(
    pn.Column(state_input,state_county_plot)
)

state_county_dash


Row
    [0] Column
        [0] Select(options=['New York, Queens County'...], value='Maryland, Calvert C...)
        [1] ParamFunction(function, _pane=Column)